# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [24]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('C:/Users/Navneet Choudhary/OneDrive/Documents/DSI/Scaling to Production/scaling_to_production/05_src/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [25]:
import pandas as pd
columns_to_exclude = ['income']
X = adult_dt.drop(columns=columns_to_exclude, axis=1)
Y = adult_dt[['income']]
Y.head()

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random state of the splitting function is 42 and it is useful to ensure reproducibility. When you split the data, setting a random state ensure the same split occurs everytime you run the code.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder


numerical_transformer = ColumnTransformer(
    transformers=[
        ('imputer', KNNImputer(n_neighbors=7, weights='distance'), ['numerical_feature_1', 'numerical_feature_2', ...]),
        ('scaler', RobustScaler(), ['numerical_feature_1', 'numerical_feature_2', ...])
    ],
    remainder='passthrough'  
)

categorical_transformer = ColumnTransformer(
    transformers=[
        ('imputer', SimpleImputer(strategy='most_frequent'), ['categorical_feature_1', 'categorical_feature_2', ...]),
        ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'), ['categorical_feature_1', 'categorical_feature_2', ...])
    ],
    remainder='passthrough'
)


full_transformer = ColumnTransformer(
    transformers=[
        ('numerical', numerical_transformer, ['numerical_feature_1', 'numerical_feature_2', ...]),
        ('categorical', categorical_transformer, ['categorical_feature_1', 'categorical_feature_2', ...])
    ],
    remainder='drop'  
)

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier  

classifier = RandomForestClassifier()


pipeline = Pipeline(steps=[
    ('preprocessor', full_transformer), 
    ('classifier', classifier)  
])


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder


numerical_features = ['numerical_feature_1', 'numerical_feature_2']
categorical_features = ['categorical_feature_1', 'categorical_feature_2']


numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
])


preprocessor = ColumnTransformer(transformers=[
    ('numerical', numerical_transformer, numerical_features),
    ('categorical', categorical_transformer, categorical_features)
])


classifier = RandomForestClassifier()

pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', classifier)
])



# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [55]:


X, y = make_classification(n_samples=1000, n_features=10, random_state=42)


numeric_features = [0, 1]  
categorical_features = [2] 

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


classifier = LogisticRegression()


pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', classifier)])


scoring = {
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}


cv_results = cross_validate(pipeline, X, y, scoring=scoring, cv=5, return_train_score=True)


print("Training Scores:")
print(f"ROC AUC: {np.mean(cv_results['train_roc_auc']):.3f}")
print(f"Accuracy: {np.mean(cv_results['train_accuracy']):.3f}")
print(f"Balanced Accuracy: {np.mean(cv_results['train_balanced_accuracy']):.3f}")
print("\nValidation Scores:")
print(f"ROC AUC: {np.mean(cv_results['test_roc_auc']):.3f}")
print(f"Accuracy: {np.mean(cv_results['test_accuracy']):.3f}")
print(f"Balanced Accuracy: {np.mean(cv_results['test_balanced_accuracy']):.3f}")


Training Scores:
ROC AUC: 0.928
Accuracy: 0.855
Balanced Accuracy: 0.854

Validation Scores:
ROC AUC: nan
Accuracy: nan
Balanced Accuracy: nan


c:\Users\Navneet Choudhary\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Navneet Choudhary\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
  File "c:\Users\Navneet Choudhary\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "c:\Users\Navneet Choudhary\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users\Navneet Choudhary\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\utils\_response.py", line 211, in _get_response_values
    

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [53]:



df_fold_results = pd.DataFrame(fold_results)


df_fold_results_sorted = df_fold_results.sort_values(by='test_neg_log_loss')

print("Fold-level results sorted by negative log loss of the test (validation) set:")
print(df_fold_results_sorted)


Fold-level results sorted by negative log loss of the test (validation) set:
   train_neg_log_loss  test_neg_log_loss  train_roc_auc  test_roc_auc  \
1            0.410513           0.508657       0.926475      0.816800   
3            0.404416           0.536008       0.928818      0.807500   
2            0.401820           0.549873       0.928093      0.789900   
0            0.400539           0.560536       0.929769      0.770877   
4            0.399058           0.565497       0.928875      0.783200   

   train_accuracy  test_accuracy  train_balanced_accuracy  \
1         0.85750          0.715                 0.857430   
3         0.85000          0.750                 0.849955   
2         0.85500          0.705                 0.854955   
0         0.85375          0.715                 0.853750   
4         0.85625          0.700                 0.856183   

   test_balanced_accuracy  
1                0.715000  
3                0.750000  
2                0.705000  
0    

Calculate the mean of each metric. 

In [51]:

df_fold_results = pd.DataFrame(fold_results)

df_fold_results_sorted = df_fold_results.sort_values(by='test_neg_log_loss')

means = df_fold_results.mean()

print("Mean metrics across all folds:")
print(means)


Mean metrics across all folds:
train_neg_log_loss         0.403269
test_neg_log_loss          0.544114
train_roc_auc              0.928406
test_roc_auc               0.793655
train_accuracy             0.854500
test_accuracy              0.717000
train_balanced_accuracy    0.854455
test_balanced_accuracy     0.716904
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [61]:

Y_pred_proba = pipe.predict_proba(X_test)

from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score


neg_log_loss = log_loss(Y_test, Y_pred_proba)

roc_auc = roc_auc_score(Y_test, Y_pred_proba[:, 1])

accuracy = accuracy_score(Y_test, Y_pred)

balanced_accuracy = balanced_accuracy_score(Y_test, Y_pred)


results = {
    'negative_log_loss': neg_log_loss,
    'roc_auc': roc_auc,
    'accuracy': accuracy,
    'balanced_accuracy': balanced_accuracy
}


print(results)



NameError: name 'pipe' is not defined

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

It was convenient because it aligns directly with how scikit-learn expects binary classification targets. It’s essential to ensure that your target variable follows the convention of having two distinct classes. This alignment simplifies the implementation and evaluation of machine learning models.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.